# Tests

## Create a test stream in redis using Aia Utilities

In [1]:
import aia_utilities as au

ru = au.Redis_Utilities()
stream = "test_stream"

## Populate it with 100 entries as timestamp advances

In [3]:
from time import sleep
from datetime import datetime
import pytz  

for i in range(100):
    sleep(0.001)  # simulate some delay
    # create a now timestamp with 6 decimals for new york time
    now = datetime.now(pytz.timezone("America/New_York"))
    timestamp = now.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
    item1 = {"timestamp": timestamp, "price": 1.0 + i, "currency": "USD"}
    ru.write(stream, item1)

ru.show()


[{'key': 'test_stream',
  'type': 'stream',
  'ttl': -1,
  'length': 100,
  'items': 100,
  'first_id': b'1757091894774-0',
  'first_entry': '{"timestamp": "2025-09-05T13:04:54.773765Z", "price": 1.0, "currency": "USD"}',
  'last_id': b'1757091894927-0',
  'last_entry': '{"timestamp": "2025-09-05T13:04:54.926992Z", "price": 100.0, "currency": "USD"}'}]

## Populate with unsorted timestamps

In [4]:
from time import sleep
from datetime import datetime, timedelta
import pytz  
import random


for i in range(100):
    # create a random timestamp from 5 minutes in the future to 5 minutes ago and insert into the stream
    future = datetime.now(pytz.timezone("America/New_York")) + timedelta(minutes=5)
    past = datetime.now(pytz.timezone("America/New_York")) - timedelta(minutes=5)
    timestamp = random.uniform(past.timestamp(), future.timestamp())
    item1 = {"timestamp": timestamp, "price": 1.0 + i, "currency": "USD"}
    ru.write(stream, item1)

ru.show()


[{'key': 'test_stream',
  'type': 'stream',
  'ttl': -1,
  'length': 200,
  'items': 200,
  'first_id': b'1757091894774-0',
  'first_entry': '{"timestamp": "2025-09-05T13:04:54.773765Z", "price": 1.0, "currency": "USD"}',
  'last_id': b'1757091897647-3',
  'last_entry': '{"timestamp": 1757091711.829953, "price": 100.0, "currency": "USD"}'}]

## Read all entries and make sure they are ordered

In [30]:
return_dict = ru.read_all('prices', order=True)

# tell me if the return_dict is ordered by timestamp or not
is_ordered = all(return_dict[i]["timestamp"] <= return_dict[i + 1]["timestamp"] for i in range(len(return_dict) - 1))
print(f"Is ordered: {is_ordered}")

print(type(return_dict))
for item in return_dict:
    print(item)

Is ordered: True
<class 'list'>
{'timestamp': '2025-09-05 15:22:10.000000', 'instrument': 'WTICO_USD', 'price': 62.45, 'bid': 62.43, 'ask': 62.47, 'spread': 0.03999999999999915, '_id': b'1757100230013-2'}
{'timestamp': '2025-09-05 15:22:15.000000', 'instrument': 'WTICO_USD', 'price': 62.445, 'bid': 62.43, 'ask': 62.46, 'spread': 0.030000000000001137, '_id': b'1757100230013-3'}
{'timestamp': '2025-09-05 15:22:30.000000', 'instrument': 'WTICO_USD', 'price': 62.44, 'bid': 62.42, 'ask': 62.46, 'spread': 0.03999999999999915, '_id': b'1757100230014-0'}
{'timestamp': '2025-09-05 15:22:35.000000', 'instrument': 'WTICO_USD', 'price': 62.435, 'bid': 62.42, 'ask': 62.45, 'spread': 0.030000000000001137, '_id': b'1757100230014-1'}
{'timestamp': '2025-09-05 15:23:20.000000', 'instrument': 'USD_CAD', 'price': 1.384605, 'bid': 1.38451, 'ask': 1.3847, 'spread': 0.0001900000000001345, '_id': b'1757100230012-0'}
{'timestamp': '2025-09-05 15:23:25.000000', 'instrument': 'USD_CAD', 'price': 1.38462, 'bid':

In [3]:

all_items = ru.read_all(stream)
print(all_items)
assert isinstance(all_items, list)
assert len(all_items) >= 2

# ensure items contain the data we wrote
prices = [i.get('price') for i in all_items]
assert 1.0 in prices and 1.1 in prices

# test read_each - start a reader that will collect two items
results = []
print(results)

def reader():
    gen = ru.read_each(stream)
    # read first two entries
    for _ in range(2):
        results.append(next(gen))

reader()

assert len(results) == 2
assert results[0]['price'] in prices
assert results[1]['price'] in prices




[{'timestamp': '2025-09-05T14:39:54.612824Z', 'price': 1.0, 'currency': 'USD', '_id': b'1757083194613-0'}, {'timestamp': '2025-09-05T14:39:54.718476Z', 'price': 2.0, 'currency': 'USD', '_id': b'1757083194718-0'}, {'timestamp': '2025-09-05T14:39:54.820821Z', 'price': 3.0, 'currency': 'USD', '_id': b'1757083194821-0'}, {'timestamp': '2025-09-05T14:39:54.925591Z', 'price': 4.0, 'currency': 'USD', '_id': b'1757083194927-0'}, {'timestamp': '2025-09-05T14:39:55.029381Z', 'price': 5.0, 'currency': 'USD', '_id': b'1757083195030-0'}, {'timestamp': '2025-09-05T14:39:55.131482Z', 'price': 6.0, 'currency': 'USD', '_id': b'1757083195132-0'}, {'timestamp': '2025-09-05T14:39:55.234697Z', 'price': 7.0, 'currency': 'USD', '_id': b'1757083195235-0'}, {'timestamp': '2025-09-05T14:39:55.337756Z', 'price': 8.0, 'currency': 'USD', '_id': b'1757083195338-0'}, {'timestamp': '2025-09-05T14:39:55.443256Z', 'price': 9.0, 'currency': 'USD', '_id': b'1757083195443-0'}, {'timestamp': '2025-09-05T14:39:55.546324Z', 

AssertionError: 

In [26]:
ru.show()

[{'key': 'prices',
  'type': 'stream',
  'ttl': -1,
  'length': 14,
  'items': 14,
  'first_id': b'1757100230012-0',
  'first_entry': '{"timestamp": "2025-09-05 15:23:20.000000", "instrument": "USD_CAD", "price": 1.384605, "bid": 1.38451, "ask": 1.3847, "spread": 0.0001900000000001345}',
  'last_id': b'1757100239521-0',
  'last_entry': '{"timestamp": "2025-09-05 15:23:59.521428", "instrument": "USD_CAD", "price": 1.38463, "bid": 1.38454, "ask": 1.38471, "spread_pips": 1.7}'}]

In [24]:
ru.delete('prices')

True

In [11]:
ru.trim('test_stream', '2025-09-05T14:55:06.612824Z')  # example timestamp

25

## Test read_each by inserting items randomly and reading them

In [ ]:
# While you read each, also insert some entries once every 10 seconds, in a separate thread
import threading
from time import sleep    
import random

ru.delete('test_stream')
def insert_entries():
    # make the range 0-5 random
    for i in range(5):
        sleep(random.randint(0, 5))
        # create a now timestamp with 6 decimals for new york time
        now = datetime.now(pytz.timezone("America/New_York"))
        timestamp = now.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
        item1 = {"timestamp": timestamp, "price": 1.0 + i, "currency": "USD"}
        ru.write(stream, item1)

# While you read each, also insert some entries once every 10 seconds, in a separate thread
thread = threading.Thread(target=insert_entries)
thread.start()

for entry in ru.read_each(stream):
    print(entry)

Exception in thread Thread-11 (insert_entries):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/threading.py", line 1041, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "/Users/code/Library/Python/3.13/lib/python/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/threading.py", line 992, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/d9/rdtj7fbj1lz5ybbbf6vstzbm0000gn/T/ipykernel_42203/2235408567.py", line 9, in insert_entries
    sleep(random.randint(0, 5))
          ^^^^^^
NameError: name 'random' is not defined. Did you forget to import 'random'?


In [6]:
from datetime import datetime, timezone

# example strings
s1 = "2000-01-01 00:00:01.123456"            # space separated
s2 = "2000-01-01T00:00:01.123456Z"           # ISO with Z

# parse and treat naive datetimes as UTC
dt1 = datetime.strptime(s1, "%Y-%m-%d %H:%M:%S.%f").replace(tzinfo=timezone.utc)
dt2 = datetime.fromisoformat(s2.replace("Z", "+00:00"))

# integer milliseconds
ms1 = int(dt1.timestamp() * 1_000)   # 1_000 ms per sec
ms2 = int(dt2.timestamp() * 1_000)

# integer microseconds
us1 = int(dt1.timestamp() * 1_000_000)
us2 = int(dt2.timestamp() * 1_000_000)

print(ms1, us1)

946684801123 946684801123456


## Is keeping the microseconds as ints 

In [7]:
# compute_years_from_micro.py
start_us = 946_684_801_123_456
JS_MAX = 2**53 - 1
REDIS_MAX = 2**63 - 1
US_PER_YEAR = 365.25 * 24 * 3600 * 1_000_000  # microseconds/year

years_to_js = (JS_MAX - start_us) / US_PER_YEAR
years_to_redis = (REDIS_MAX - start_us) / US_PER_YEAR

print("years to JS safe-int:", years_to_js)
print("years to Redis 64-bit:", years_to_redis)


years to JS safe-int: 255.42228983248202
years to Redis 64-bit: 292241.024414203


In [8]:
# current (works, a bit heavier)
from datetime import datetime
ts_ms = int(datetime.utcnow().timestamp() * 1000)
print(ts_ms)

# faster: integer ms using time.time_ns()
import time
ts_ms = time.time_ns() // 1_000_000
print(ts_ms)

# also OK: float-based (slightly slower than time_ns)
ts_ms = int(time.time() * 1000)
print(ts_ms)

1757107243774
1757092843774
1757092843774


/var/folders/d9/rdtj7fbj1lz5ybbbf6vstzbm0000gn/T/ipykernel_42236/3226634443.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts_ms = int(datetime.utcnow().timestamp() * 1000)
